In [ ]:
using Plots
using FFTW

In [ ]:
# Reflect index as -i&(n - 1), but for 1-based indexes
flip_idx(i, n) = -(i - 1)&(n - 1) + 1

x = [681, 683, 414, 987, 336, 583, 121, 772]
y = [354, 929, 058, 303, 314, 552, 248, 280]
N = length(x)

round.([fft(x) fft(y)])[1:N÷2 + 1, :]

In [ ]:
# Calculate simultaneous real FFTs
n = N÷2 + 1
X, Y = zeros(Complex, n), zeros(Complex, n)

# Combine x and y, then calculate it's FFT
XY = fft(x + y*im)/2

# Now separate the two using even/odd symmetry property
for i in 1:n
    s = XY[i]
    t = conj(XY[flip_idx(i, N)])
    X[i] = (s + t)
    Y[i] = (s - t)*im
end

round.([X Y])

In [ ]:
# Calculate real valued FFT
X = zeros(Complex, N÷2 + 1)

# Calculate FFTs of evens and odds simultaneously
xe, xo = x[1:2:N], x[2:2:N]
Xeo = fft(xe + xo*im)/2

w, wm = -im, cispi(-2/N)
for i in 1:N÷4 + 1
    p = Xeo[i]
    q = conj(Xeo[flip_idx(i, N÷2)])
    Xei = (p + q)
    Xowi = (p - q)*w
    X[i] = Xei + Xowi
    X[N÷2 - i + 2] = conj(Xei - Xowi)
    w *= wm
end

round.([X rfft(x)])

In [ ]:
# Calculate real valued ifft
X = rfft(x)
Xeo = zeros(ComplexF64, N÷2)

w, wm = im, cispi(2/N)
for i in 1:N÷4 + 1
    p = X[i]
    q = conj(X[N÷2 - i + 2])
    Xei = (p + q)
    Xoi = (p - q)*w
    Xeo[i] = conj(Xei + Xoi)
    Xeo[flip_idx(i, N÷2)] = (Xei - Xoi)
    w *= wm
end
xeo = fft(Xeo)/N
[x reshape(transpose([real(xeo) -imag(xeo)]), N)]